In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import joblib
import json
import os
from datetime import datetime


from ensemble_modelling import select_feature_columns, select_target_columns, walk_forward_train_ensemble_model

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = "ensemble_tbm10d" # ИМЯ МОДЕЛИ

In [3]:
# ==============================================================================
# ОСНОВНОЙ СКРИПТ МОДЕЛИРОВАНИЯ
# ==============================================================================

# --- ШАГ 1: Подготовка данных ---

data_folder = "../data/"
final_dataset_filename = 'moex_final_dataset.csv'
TARGET_COLUMN = 'tbm_10d' # Наша цель для предсказания!

# Загружаем финальный датасет
print(f"Загрузка данных из: {data_folder + final_dataset_filename}")
try:
    df = pd.read_csv(data_folder + final_dataset_filename)
    # if features_to_remove:
    #     df = df.drop(columns=features_to_remove)
    #     print(f"Удалено {len(features_to_remove)} неважных признаков.")


    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    print("Данные успешно загружены.")
except FileNotFoundError:
    print("ОШИБКА: Финальный датасет не найден.")
    raise


print(f"\nИсходный размер датасета: {len(df)} строк.")
df.dropna(subset=[TARGET_COLUMN], inplace=True)
df.reset_index(drop=True, inplace=True)
print(f"Размер датасета после фильтрации по CUSUM-событиям: {len(df)} строк.")


Загрузка данных из: ../data/moex_final_dataset.csv
Данные успешно загружены.

Исходный размер датасета: 15143 строк.
Размер датасета после фильтрации по CUSUM-событиям: 15143 строк.


In [4]:
feature_cols = select_feature_columns(df)

X = df[feature_cols]
y = df[TARGET_COLUMN].astype(int)




print(f"\nЦелевая переменная (y): {TARGET_COLUMN}")
print(f"Количество признаков (X): {len(feature_cols)}")
print("Признаки:", feature_cols)


Целевая переменная (y): tbm_10d
Количество признаков (X): 245
Признаки: ['roc_10', 'low_to_sma_30', 'zscore_close_20', 'sma3_cross_sma7', 'rolling_skew_14', 'range_expansion_ratio', 'willr_21', 'close_to_ema_50', 'MACDh_12_26_9', 'cbr_rate_change_value', 'sma_7', 'fft_abs_5', 'relative_turnover_7', 'high_to_sma_15', 'rsi_21', 'momentum_rank_100d', 'fft_abs_8', 'momentum_rank_21d', 'bb_upper_breakout', 'open_to_sma_20', 'sma50_above_sma200', 'open_to_sma_7', 'atr_14', 'relative_volume_20', 'day_range_norm', 'open_to_sma_50', 'sma20_cross_sma50', 'PVO_12_26_9', 'sma_40', 'relative_turnover_30', 'wv_L3_std', 'quarter', 'momentum_rank_5d', 'rolling_kurt_14', 'roc_20', 'rsi_30', 'momentum_sharpe_14d', 'adx_trend_strength', 'MACD_12_26_9', 'stl_seasonal_last', 'low_to_sma_70', 'momentum_sharpe_3d', 'fft_abs_4', 'close_to_sma_7', 'rolling_skew_7', 'close_to_sma_10', 'kama_30', 'close_to_sma_3', 'momentum_sharpe_7d', 'bb_lower_breakout', 'wv_L2_std', 'close_to_sma_30', 'sma70_cross_sma200', '

In [5]:
walk_forward_train_ensemble_model(df, X, y)


Начинаем Walk-Forward валидацию с 5 сплитами...

--- Сплит 1/5 ---
Размер обучающей выборки: 2528
Размер тестовой выборки: 2523
Обучение модели...
Оценка модели...

Точность (Accuracy) на сплите 1: 0.4998
Детальный отчет по качеству (Classification Report):
              precision    recall  f1-score   support

   -1 (Sell)       0.36      0.25      0.29       720
    0 (Hold)       0.65      0.44      0.53       684
     1 (Buy)       0.50      0.70      0.58      1119

    accuracy                           0.50      2523
   macro avg       0.50      0.46      0.47      2523
weighted avg       0.50      0.50      0.48      2523

📊 ROC AUC -1:   0.5977575953657485
📊 ROC AUC 0:   0.7656955057573241
📊 ROC AUC 1:   0.6010339410696872

--- Сплит 2/5 ---
Размер обучающей выборки: 5051
Размер тестовой выборки: 2523
Обучение модели...
Оценка модели...

Точность (Accuracy) на сплите 2: 0.4717
Детальный отчет по качеству (Classification Report):
              precision    recall  f1-score   s